<a href="https://colab.research.google.com/github/JS-Choi513/Predicting-the-Housing-Price-Index-in-Changwon-City-Using-the-LSTM-Model/blob/main/LSTM_Price_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!df -h
!cat /proc/meminfo
!cat /proc/cpuinfo
!nvidia-smi

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# 데이터셋 업로드 
from google.colab import files

#uploaded = files.upload() 

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))  
data = pd.read_csv('/content/uc_housing_price.csv')
data['new_Date'] = pd.to_datetime(data['date'])
print(data.head())
print('\n')
print(data.info())
print('\n')
print(type(data['new_Date'][0]))
data.drop('date', axis = 1, inplace=True)
data.set_index('new_Date', inplace=True)
print(data.head())
print('\n')
print(data.info())
#df = pd.DataFrame(data)

In [ ]:
data['price'].plot()

In [ ]:
split_date = pd.Timestamp('01-01-2017')
#split_date[0:6]
#date =str(split_date)[0:7] 
#print(date)



In [ ]:
train = df.loc[:split_date, ['price']]
test = df.loc[split_date:, ['price']]

ax = train.plot()
test.plot(ax=ax)
plt.legend(['train', 'test'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Min-Max scaleing
sc = MinMaxScaler()# 훈련데이터 분포 추정 
# 훈련데이터 변환 
train_sc = sc.fit_transform(train)
#테스트 데이터 변환 
test_sc = sc.transform(test)

train_sc

In [ ]:
train_sc_df = pd.DataFrame(train_sc, columns=['price'], index=train.index)
test_sc_df = pd.DataFrame(test_sc, columns=['price'], index=test.index)
train_sc_df.head()

In [ ]:
# LSTM 슬라이딩 윈도우 구성 과거 1년(12개월) 
for s in range(1, 13): 
    train_sc_df['shift_{}'.format(s)] = train_sc_df['price'].shift(s)
    test_sc_df['shift_{}'.format(s)] = test_sc_df['price'].shift(s)

train_sc_df.head(13)

In [ ]:
#dropna 함수로 빈값제거 
X_train = train_sc_df.dropna().drop('price', axis=1)
y_train = train_sc_df.dropna()[['price']]

X_test = test_sc_df.dropna().drop('price', axis=1)
y_test = test_sc_df.dropna()[['price']]
X_train.head()

In [ ]:
print(type(X_train))
X_train = X_train.values
print(type(X_train))
X_test= X_test.values
y_train = y_train.values
y_test = y_test.values
 

print(X_train.shape)
print(y_train.shape)
#훈련데이터 x_train 은 145개의 데이터와 12개의 슬롯을 가짐 
#훈련값 y_train은 145개의 데이터와 1개의 타겟값을 가짐 

In [ ]:
#LSTM 학습을 위한 3차원 데이터 변환, size, time stamp, feature
X_train_t = X_train.reshape(X_train.shape[0], 12, 1)
X_test_t = X_test.reshape(X_test.shape[0], 12, 1)

print("최종 DATA")
print(X_train_t.shape)
print(X_train_t)
print(y_train)

In [ ]:
from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
import keras.backend as K 
from keras.callbacks import EarlyStopping

K.clear_session()
    
model = Sequential() # Sequeatial Model 
model.add(LSTM(20, input_shape=(12, 1))) # (timestep, feature) #20개의 메모리셀을 가짐, 입력데이터 모델 정보 12타임스택, 1피터 
model.add(Dense(1)) # output = 1 #완전연결 레이어 예측하고자 하는 값이 하나이므로 1 
model.compile(loss='mean_squared_error', optimizer='adam') 
#컴파일 호출으로 학습과정 설정, 훈련과정 설정,loss함수 설정 오차제곱평균 사용 
model.summary()
#모델구성 확인 

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)
# 모델 훈련(훈련데이터, 지도학습 데티어, 에폭: 전체입력데이터 순회 횟수
# 100이면 전체 데이터를 100번 사용해서 학습 
#, 배치 사이즈(모델이 데이터를 작은 배치로 나누고 훈련과정에서 이 배치를 순회)
# verbose 훈련진행과정 보여줌 1일경우 진행막대 출력, 2일 경우 배치마다 손실정보 출력 
model.fit(X_train_t, y_train, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

In [ ]:
score = model.evaluate(X_test_t, y_test, batch_size=30)
print(score)

In [ ]:
train_sc_df.describe()
test_sc_df.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


y_pred = model.predict(X_test_t, batch_size=32)
plt.scatter(y_test, y_pred)
plt.xlabel("Price Index: $Y_i$")
plt.ylabel("Predicted price Index: $\hat{Y}_i$")
plt.title("Prices vs Predicted price Index: $Y_i$ vs $\hat{Y}_i$")